##### Copyright 2018 Die TensorFlow-Autoren.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Trainingskontrollpunkte

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ansicht auf TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Führen Sie in Google Colab aus</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Quelle auf GitHub anzeigen</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Notizbuch herunterladen</a></td>
</table>

Der Ausdruck "Speichern eines TensorFlow-Modells" bedeutet normalerweise eines von zwei Dingen:

1. Checkpoints ODER
2. SavedModel.

Checkpoints erfassen den genauen Wert aller von einem Modell verwendeten Parameter ( `tf.Variable` Objekte). Prüfpunkte enthalten keine Beschreibung der vom Modell definierten Berechnung und sind daher normalerweise nur dann nützlich, wenn Quellcode verfügbar ist, der die gespeicherten Parameterwerte verwendet.

Das SavedModel-Format enthält andererseits zusätzlich zu den Parameterwerten (Prüfpunkt) eine serialisierte Beschreibung der vom Modell definierten Berechnung. Modelle in diesem Format sind unabhängig vom Quellcode, mit dem das Modell erstellt wurde. Sie eignen sich daher für die Bereitstellung über TensorFlow Serving, TensorFlow Lite, TensorFlow.js oder Programme in anderen Programmiersprachen (C, C ++, Java, Go, Rust, C # usw. TensorFlow-APIs).

Dieses Handbuch behandelt APIs zum Schreiben und Lesen von Prüfpunkten.

## Konfiguration

In [0]:
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [0]:
net = Net()

## Speichern von `tf.keras` Trainings-APIs

See the [`tf.keras` guide on saving and restoring](./keras/overview.ipynb#save_and_restore).

`tf.keras.Model.save_weights` speichert einen TensorFlow-Prüfpunkt. 

In [0]:
net.save_weights('easy_checkpoint')

## Checkpoints schreiben


The persistent state of a TensorFlow model is stored in `tf.Variable` objects. These can be constructed directly, but are often created through high-level APIs like `tf.keras.layers` or `tf.keras.Model`.

Der einfachste Weg, Variablen zu verwalten, besteht darin, sie an Python-Objekte anzuhängen und dann auf diese Objekte zu verweisen.

Unterklassen von `tf.train.Checkpoint` , `tf.keras.layers.Layer` und `tf.keras.Model` automatisch Variablen, die ihren Attributen zugewiesen sind. Im folgenden Beispiel wird ein einfaches lineares Modell erstellt und anschließend Prüfpunkte geschrieben, die Werte für alle Variablen des Modells enthalten.

You can easily save a model-checkpoint with `Model.save_weights`

### Manuelle Checkpointing

#### Konfiguration

Um alle Funktionen von `tf.train.Checkpoint` definieren Sie einen Spielzeugdatensatz und einen Optimierungsschritt:

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [0]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### Erstellen Sie die Prüfpunktobjekte

To manually make a checkpoint you will need a `tf.train.Checkpoint` object. Where the objects you want to checkpoint are set as attributes on the object.

Ein `tf.train.CheckpointManager` kann auch hilfreich sein, um mehrere Prüfpunkte zu verwalten.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### Trainiere und überprüfe das Modell

Die folgende Trainingsschleife erstellt eine Instanz des Modells und eines Optimierers und sammelt sie dann in einem `tf.train.Checkpoint` Objekt. Es ruft den Trainingsschritt in einer Schleife für jeden Datenstapel auf und schreibt regelmäßig Prüfpunkte auf die Festplatte.

In [0]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [0]:
train_and_checkpoint(net, manager)

#### Wiederherstellen und Training fortsetzen

Nach dem ersten können Sie ein neues Modell und einen neuen Manager übergeben, aber das Training genau dort aufnehmen, wo Sie aufgehört haben:

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

Das Objekt `tf.train.CheckpointManager` löscht alte Prüfpunkte. Oben ist es so konfiguriert, dass nur die drei neuesten Prüfpunkte beibehalten werden.

In [0]:
print(manager.checkpoints)  # List the three remaining checkpoints

Diese Pfade, z. B. `'./tf_ckpts/ckpt-10'` , sind keine Dateien auf der Festplatte. Stattdessen sind sie Präfixe für eine `index` und eine oder mehrere Datendateien, die die Variablenwerte enthalten. Diese Präfixe sind in einer einzigen `checkpoint` ( `'./tf_ckpts/checkpoint'` ) zusammengefasst, in der der `CheckpointManager` seinen Status speichert.

In [0]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## Lademechanik

TensorFlow matches variables to checkpointed values by traversing a directed graph with named edges, starting from the object being loaded. Edge names typically come from attribute names in objects, for example the `"l1"` in `self.l1 = tf.keras.layers.Dense(5)`. `tf.train.Checkpoint` uses its keyword argument names, as in the `"step"` in `tf.train.Checkpoint(step=...)`.

Das Abhängigkeitsdiagramm aus dem obigen Beispiel sieht folgendermaßen aus:

![Visualisierung des Abhängigkeitsgraphen für die Beispieltrainingsschleife](https://tensorflow.org/images/guide/whole_checkpoint.svg)

Mit dem Optimierer in Rot, regulären Variablen in Blau und Optimierungssteckplatzvariablen in Orange. Die anderen Knoten, die beispielsweise den `tf.train.Checkpoint` , sind schwarz.

Slot-Variablen sind Teil des Optimierungsstatus, werden jedoch für eine bestimmte Variable erstellt. Zum Beispiel entsprechen die `'m'` Kanten oben dem Impuls, den der Adam-Optimierer für jede Variable verfolgt. Steckplatzvariablen werden nur dann in einem Prüfpunkt gespeichert, wenn sowohl die Variable als auch der Optimierer gespeichert würden, also die gestrichelten Kanten.

Durch Aufrufen von `restore()` für ein `tf.train.Checkpoint` Objekt werden die angeforderten Wiederherstellungen in die Warteschlange gestellt und Variablenwerte wiederhergestellt, sobald ein übereinstimmender Pfad vom `Checkpoint` Objekt vorhanden ist. Zum Beispiel können wir nur die Vorspannung aus dem oben definierten Modell laden, indem wir einen Pfad durch das Netzwerk und die Schicht rekonstruieren.

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # We get the restored value now

Das Abhängigkeitsdiagramm für diese neuen Objekte ist ein viel kleinerer Teilgraph des größeren Prüfpunkts, den wir oben geschrieben haben. Es enthält nur den Bias und einen Sicherungszähler, mit dem `tf.train.Checkpoint` Checkpoints nummeriert.

![Visualisierung eines Untergraphen für die Bias-Variable](https://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore()` gibt ein Statusobjekt zurück, das optionale Zusicherungen enthält. Alle Objekte , die wir in unserem neuen erstellten `Checkpoint` wurden restauriert, so `status.assert_existing_objects_matched()` übergibt.

In [0]:
status.assert_existing_objects_matched()

There are many objects in the checkpoint which haven't matched, including the layer's kernel and the optimizer's variables. `status.assert_consumed()` only passes if the checkpoint and the program match exactly, and would throw an exception here.

### Verzögerte Restaurationen

`Layer` in TensorFlow können die Erstellung von Variablen bis zum ersten Aufruf verzögern, wenn Eingabeformen verfügbar sind. Beispielsweise hängt die Form des Kernels einer `Dense` Ebene sowohl von der Eingabe- als auch von der Ausgabeform der Ebene ab. Daher reicht die als Konstruktorargument erforderliche Ausgabeform nicht aus, um die Variable selbst zu erstellen. Da beim Aufrufen einer `Layer` auch der Wert der Variablen gelesen wird, muss zwischen der Erstellung der Variablen und ihrer ersten Verwendung eine Wiederherstellung erfolgen.

Um diese Redewendung zu unterstützen, werden in `tf.train.Checkpoint` Warteschlangen Wiederherstellungen durchgeführt, für die noch keine übereinstimmende Variable vorhanden ist.

In [0]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restored

### Manuelle Überprüfung der Kontrollpunkte

`tf.train.list_variables` listet die Prüfpunktschlüssel und -formen von Variablen in einem Prüfpunkt auf. Checkpoint-Schlüssel sind Pfade in der oben gezeigten Grafik.

In [0]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

### Listen- und Wörterbuchverfolgung

Wie bei direkten Attributzuweisungen wie `self.l1 = tf.keras.layers.Dense(5)` wird durch das Zuweisen von Listen und Wörterbüchern zu Attributen deren Inhalt verfolgt.

In [0]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

Möglicherweise bemerken Sie Wrapper-Objekte für Listen und Wörterbücher. Diese Wrapper sind checkpointable Versionen der zugrunde liegenden Datenstrukturen. Genau wie beim attributbasierten Laden stellen diese Wrapper den Wert einer Variablen wieder her, sobald sie dem Container hinzugefügt wird.

In [0]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

The same tracking is automatically applied to subclasses of `tf.keras.Model`, and may be used for example to track lists of layers.

## Speichern objektbasierter Prüfpunkte mit Estimator

Siehe den [Estimator-Leitfaden](https://www.tensorflow.org/guide/estimator) .

Estimators by default save checkpoints with variable names rather than the object graph described in the previous sections. `tf.train.Checkpoint` will accept name-based checkpoints, but variable names may change when moving parts of a model outside of the Estimator's `model_fn`. Saving object-based checkpoints makes it easier to train a model inside an Estimator and then use it outside of one.

In [0]:
import tensorflow.compat.v1 as tf_compat

In [0]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Tell the Estimator to save "ckpt" in an object-based format.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

`tf.train.Checkpoint` kann dann die Prüfpunkte des Schätzers aus seinem `model_dir` .

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # From est.train(..., steps=10)

## Zusammenfassung

TensorFlow-Objekte bieten einen einfachen automatischen Mechanismus zum Speichern und Wiederherstellen der Werte der von ihnen verwendeten Variablen.
